# PyTorch Assignment: Multi-Layer Perceptron (MLP)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: 

### Multi-Layer Perceptrons

The simple logistic regression example we went over in the previous notebook is essentially a one-layer neural network, projecting straight from the input to the output predictions.
While this can be effective for linearly separable data, occasionally a little more complexity is necessary.
Neural networks with additional layers are typically able to learn more complex functions, leading to better performance.
These additional layers (called "hidden" layers) transform the input into one or more intermediate representations before making a final prediction.

In the logistic regression example, the way we performed the transformation was with a fully-connected layer, which consisted of a linear transform (matrix multiply plus a bias).
A neural network consisting of multiple successive fully-connected layers is commonly called a Multi-Layer Perceptron (MLP). 
In the simple MLP below, a 4-d input is projected to a 5-d hidden representation, which is then projected to a single output that is used to make the final prediction.

<img src="Figures/MLP.png" width="300"/>

For the assignment, you will be building a MLP for MNIST.
Mechanically, this is done very similary to our logistic regression example, but instead of going straight to a 10-d vector representing our output predictions, we might first transform to a 500-d vector with a "hidden" layer, then to the output of dimension 10.
Before you do so, however, there's one more important thing to consider.

### Nonlinearities

We typically include nonlinearities between layers of a neural network.
There's a number of reasons to do so.
For one, without anything nonlinear between them, successive linear transforms (fully connected layers) collapse into a single linear transform, which means the model isn't any more expressive than a single layer.
On the other hand, intermediate nonlinearities prevent this collapse, allowing neural networks to approximate more complex functions.

There are a number of nonlinearities commonly used in neural networks, but one of the most popular is the [rectified linear unit (ReLU)](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)):

\begin{align}
x = \max(0,x)
\end{align}

There are a number of ways to implement this in PyTorch.
We could do it with elementary PyTorch operations:

In [1]:
import torch

x = torch.rand(5, 3)*2 - 1
x_relu_max = torch.max(torch.zeros_like(x),x)

print("x: {}".format(x))
print("x after ReLU with max: {}".format(x_relu_max))

x: tensor([[ 0.0229,  0.2082, -0.1367],
        [ 0.2571, -0.1594,  0.1808],
        [-0.0476, -0.6198,  0.4700],
        [-0.8136, -0.9273,  0.4569],
        [ 0.0589,  0.8808, -0.2375]])
x after ReLU with max: tensor([[0.0229, 0.2082, 0.0000],
        [0.2571, 0.0000, 0.1808],
        [0.0000, 0.0000, 0.4700],
        [0.0000, 0.0000, 0.4569],
        [0.0589, 0.8808, 0.0000]])


Of course, PyTorch also has the ReLU implemented, for example in `torch.nn.functional`:

In [2]:
import torch.nn.functional as F

x_relu_F = F.relu(x)

print("x after ReLU with nn.functional: {}".format(x_relu_F))

x after ReLU with nn.functional: tensor([[0.0229, 0.2082, 0.0000],
        [0.2571, 0.0000, 0.1808],
        [0.0000, 0.0000, 0.4700],
        [0.0000, 0.0000, 0.4569],
        [0.0589, 0.8808, 0.0000]])


Same result.

### Assignment

Build a 2-layer MLP for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image (784 dimensions) ->  
fully connected layer (500 hidden units) -> nonlinearity (ReLU) ->  
fully connected (10 hidden units) -> softmax

Try building the model both with basic PyTorch operations, and then again with more object-oriented higher-level APIs. 
You should get similar results!


*Some hints*:
- Even as we add additional layers, we still only require a single optimizer to learn the parameters.
Just make sure to pass all parameters to it!
- As you'll calculate in the Short Answer, this MLP model has many more parameters than the logisitic regression example, which makes it more challenging to learn.
To get the best performance, you may want to play with the learning rate and increase the number of training epochs.
- Be careful using `torch.nn.CrossEntropyLoss()`. 
If you look at the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#crossentropyloss): you'll see that `torch.nn.CrossEntropyLoss()` combines the softmax operation with the cross-entropy.
This means you need to pass in the logits (predictions pre-softmax) to this loss.
Computing the softmax separately and feeding the result into `torch.nn.CrossEntropyLoss()` will significantly degrade your model's performance!

In [33]:
### YOUR CODE HERE
import numpy as np
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms
from tqdm.notebook import tqdm

mnist_train = datasets.MNIST(root="./datasets", train=True, transform=transforms.ToTensor(), download=True)
mnist_test = datasets.MNIST(root="./datasets", train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=100, shuffle=False)

# Dimensions
input_dim = mnist_train.train_data.shape[1] * mnist_train.train_data.shape[2] #input dimensions
L1_o_dim = 500
L2_o_dim = 10
# Initialize parameters
# Layer 1
W1 = torch.randn(input_dim, L1_o_dim)/np.sqrt(input_dim)
W1.requires_grad_()
b1 = torch.zeros(L1_o_dim, requires_grad=True)
# Layer 2
W2 = torch.randn(L1_o_dim, L2_o_dim)/np.sqrt(L1_o_dim)
W2.requires_grad_()
b2 = torch.zeros(L2_o_dim, requires_grad=True)

# Optimizer
optimizer = torch.optim.SGD([W1,b1,W2,b2], lr=0.1)

# Training
for images, labels in tqdm(train_loader):
    # Reset the gradients
    optimizer.zero_grad()
    # Forward
    # Input
    X = images.view(-1, input_dim)
    # Layer 1
    Z1 = X @ W1 + b1
    A1 = F.relu(Z1)
    # Layer 2
    Z2 = A1 @ W2 + b2
    # Loss: Softmax with cross_entropy
    loss = F.cross_entropy(Z2, labels)
    
    # Backward
    loss.backward()
    optimizer.step()

# Test validation set
correct = 0
total = len(mnist_test)
with torch.no_grad():
    for images, labels in tqdm(test_loader):
        # Forward
        # Input
        X = images.view(-1, input_dim)
        # Layer 1
        Z1 = X @ W1 + b1
        A1 = F.relu(Z1)
        # Layer 2
        Z2 = A1 @ W2 + b2
        
        predictions = torch.argmax(Z2, dim=1)
        correct += torch.sum((predictions == labels).float())

print(f'Test accuracy: {correct/total}')
# Make sure to print out your accuracy on the test set at the end.


Test accuracy: 0.928600013256073


### Short answer
How many trainable parameters does your model have? 
How does this compare to the logisitic regression example?

The 2-layer model has: 
Layer 1: (784 * 500) + 500 = 392500
Layer 2: (500 * 10) + 10 = 5010
Total of: <b>397510 parameters</b>

The logistic regression example has 784 * 10 + 10 = 7850 parameters

The 2-layer model has <b>389660 more parameters</b> than the logistic regression